In [34]:
import pandas as pd
import seaborn as sns
import pycaret as pyc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from pycaret.classification import *


le = LabelEncoder()

CLASSIFICATION_TARGET = 'h1n1_vaccine'

data = pd.read_csv('../../data/training_set_features.csv')
target = pd.read_csv('../../data/training_set_labels.csv')[CLASSIFICATION_TARGET]
df = data.join(target, how='inner')
# train, test = train_test_split(df, shuffle=False)

df = df.astype('category')
for col in df:
    df[col] = df[col].cat.codes


### Fitting PyCaret models

In [37]:
clf = setup(data=df, target='h1n1_vaccine',session_id=123, normalize=True, fix_imbalance=True, html=True, feature_selection=True)


Setup Succesfully Completed!


,Description,Value
0,session_id,123
1,Target Type,Binary
2,Label Encoded,None
3,Original Data,"(26707, 37)"
4,Missing Values,False
5,Numeric Features,36
6,Categorical Features,0
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [38]:
df = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
0,Light Gradient Boosting Machine,0.8275,0.8365,0.5609,0.6010,0.5797,0.4714,0.4722,0.6189
1,CatBoost Classifier,0.8275,0.8298,0.5483,0.6034,0.5740,0.4663,0.4675,9.8636
2,Extra Trees Classifier,0.8266,0.8252,0.5152,0.6081,0.5576,0.4508,0.4533,0.8536
3,Extreme Gradient Boosting,0.8205,0.8239,0.5389,0.5841,0.5601,0.4477,0.4485,3.0860
4,Gradient Boosting Classifier,0.8161,0.8228,0.5801,0.5657,0.5723,0.4553,0.4557,3.4083
5,Random Forest Classifier,0.8116,0.7913,0.4567,0.5701,0.5068,0.3922,0.3961,0.1125
6,Ada Boost Classifier,0.7797,0.7827,0.5540,0.4842,0.5164,0.3747,0.3764,0.8587
7,Ridge Classifier,0.7512,0.0000,0.6497,0.4419,0.5258,0.3654,0.3780,0.0180
8,Linear Discriminant Analysis,0.7512,0.7904,0.6497,0.4419,0.5258,0.3654,0.3780,0.0857
9,Logistic Regression,0.7396,0.7803,0.6525,0.4262,0.5155,0.3480,0.3629,0.2121
